Assignment 2\
Name: Tong Wai Ho\
Sid: 54812369

Question 3 - Data Querying

In [7]:
# #a)
# SELECT
#     h.hacker_id,
#     h.name,
#     COUNT(DISTINCT s.task_id) AS num_tasks
# FROM
#     Hackers h
#     JOIN Submissions s ON h.hacker_id = s.hacker_id
# GROUP BY
#     h.hacker_id, h.name
# HAVING
#     COUNT(DISTINCT s.task_id) >
#         (
#             SELECT AVG(task_count)
#             FROM (
#                 SELECT COUNT(DISTINCT s2.task_id) AS task_count
#                 FROM Submissions s2
#                 GROUP BY s2.hacker_id
#             ) AS sub
#         )
# ORDER BY
#     num_tasks DESC,
#     h.hacker_id ASC;

In [6]:
# #b)
# SELECT
#     t.task_id,
#     t.description,
#     COUNT(top3.hacker_id) * t.bonus AS total_bonus_awarded
# FROM
#     Tasks t
#     LEFT JOIN (
#         SELECT
#             s.task_id,
#             s.hacker_id
#         FROM (
#             SELECT
#                 s1.task_id,
#                 s1.hacker_id,
#                 DENSE_RANK() OVER (PARTITION BY s1.task_id ORDER BY s1.score DESC, s1.submission_id ASC) AS rnk
#             FROM
#                 Submissions s1
#         ) ranked
#         WHERE
#             ranked.rnk <= 3
#         GROUP BY
#             ranked.task_id,
#             ranked.hacker_id
#     ) top3
#     ON t.task_id = top3.task_id
# GROUP BY
#     t.task_id, t.description, t.bonus
# ORDER BY
#     t.task_id ASC;

In [8]:
# #c)
# SELECT
#     s.submission_id,
#     s.hacker_id,
#     h.name,
#     s.score
# FROM
#     Submissions s
#     JOIN Hackers h ON s.hacker_id = h.hacker_id
# WHERE
#     s.submission_date = '2023-01-01'
#     AND s.submission_id = (
#         SELECT MIN(sub2.submission_id)
#         FROM Submissions sub2
#         WHERE
#             sub2.task_id = s.task_id
#             AND sub2.submission_date = '2023-01-01'
#             AND sub2.score = (
#                 SELECT MAX(sub3.score)
#                 FROM Submissions sub3
#                 WHERE
#                     sub3.task_id = s.task_id
#                     AND sub3.submission_date = '2023-01-01'
#             )
#     )
# ORDER BY
#     s.task_id ASC;

In [9]:
# #d)
# SELECT
#     h.hacker_id,
#     h.name,
#     SUM(best_scores.max_score) AS total_score,
#     COUNT(best_scores.task_id) AS num_tasks,
#     ROUND(SUM(best_scores.max_score) * 1.0 / COUNT(best_scores.task_id), 2) AS avg_score_per_task
# FROM
#     Hackers h
#     JOIN (
#         SELECT
#             s.hacker_id,
#             s.task_id,
#             MAX(s.score) AS max_score
#         FROM
#             Submissions s
#         GROUP BY
#             s.hacker_id, s.task_id
#     ) best_scores
#     ON h.hacker_id = best_scores.hacker_id
# GROUP BY
#     h.hacker_id, h.name
# ORDER BY
#     total_score DESC,
#     h.hacker_id ASC;

In [10]:
# #e)
# SELECT
#     h.hacker_id,
#     h.name,
#     h.bank_account
# FROM
#     Hackers h
# LEFT JOIN
#     Submissions s ON h.hacker_id = s.hacker_id
# WHERE
#     s.hacker_id IS NULL;

In [5]:
#Question 4 - Recommendation System

In [88]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [89]:
# Load the dataset into a pandas dataframe
base = Path.cwd()   # or Path('.') 
goodbooks_dir = base / "data" / "goodbooks"
ratings = pd.read_csv(f'{goodbooks_dir}/ratings.csv')
books = pd.read_csv(f'{goodbooks_dir}/books.csv')

In [90]:
# Data Cleansing
books.drop_duplicates(inplace=True)
ratings.drop_duplicates(inplace=True)

In [91]:
#Limit the data to top 5,000 users and top 3,000 books based onthe number of interactions (ratings)
top_users = ratings['user_id'].value_counts().head(5000).index
top_books = ratings['book_id'].value_counts().head(3000).index
filtered_ratings = ratings[ratings['user_id'].isin(top_users) & ratings['book_id'].isin(top_books)]

In [92]:
filtered_ratings = filtered_ratings.drop_duplicates(subset=['user_id', 'book_id'])

In [93]:
# Verify and then split
print("filtered_ratings shape:", filtered_ratings.shape)
train_data, test_data = train_test_split(filtered_ratings, test_size=0.15, random_state=42)

filtered_ratings shape: (105087, 3)


In [ ]:
# Create User-Item Matrix from train data
user_item_matrix = train_data.pivot_table(index='user_id', columns='book_id', values='rating').fillna(0)

# Compute User Similarity Matrix (User-based CF)
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Get top similar users
def get_top_similar_users(target_user_id, top_n=10):
    user_scores = user_similarity_df.loc[target_user_id]
    top_users = user_scores.sort_values(ascending=False).iloc[1:top_n+1]
    return top_users.index.tolist()

# Recommend Top-N Books for each user in test set
def recommend_books_for_user(target_user_id, N=10):
    # Get user's training books
    training_books = set(train_data[train_data['user_id'] == target_user_id]['book_id'])
    top_users = get_top_similar_users(target_user_id)
    # Get ratings from similar users
    candidate_books = defaultdict(float)
    for similar_user in top_users:
        sim_score = user_similarity_df.loc[target_user_id, similar_user]
        similar_user_ratings = train_data[train_data['user_id'] == similar_user]
        for _, row in similar_user_ratings.iterrows():
            if row['book_id'] not in training_books:  # Recommend only unseen books
                candidate_books[row['book_id']] += row['rating'] * sim_score
    # Sort candidates by score & return top N
    recommended = sorted(candidate_books, key=candidate_books.get, reverse=True)[:N]
    return recommended

# Evaluation: Hit Rate and F1 Score
hits, total, true_positives, false_positives = 0, 0, 0, 0
for user in test_data['user_id'].unique():
    # Books user rated in the test set (ground truth)
    test_books = set(test_data[test_data['user_id'] == user]['book_id'])
    if not test_books:
        continue
    recommended_books = set(recommend_books_for_user(user, N=10))
    hit = len(test_books & recommended_books) > 0
    hits += hit
    total += 1
    tp = len(test_books & recommended_books)
    fp = len(recommended_books - test_books)
    fn = len(test_books - recommended_books)
    true_positives += tp
    false_positives += fp

# Metrics
hit_rate = hits / total if total > 0 else 0
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + fn) if (true_positives + fn) > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"Top-10 Hit Rate: {hit_rate:.4f}")
print(f"F1 Score: {f1_score:.4f}")

Top-10 Hit Rate: 0.4441
F1 Score: 0.1417


In [95]:
# Display top-10 recommendations for a sample user in the test set
sample_user = test_data['user_id'].iloc[0]
recommended_books = recommend_books_for_user(sample_user, N=10)
print(f"Top-10 recommended books for user {sample_user}: {recommended_books}")

Top-10 recommended books for user 51243: [np.int64(6368), np.int64(6374), np.int64(4712), np.int64(4396), np.int64(5805), np.int64(8332), np.int64(3562), np.int64(4775), np.int64(5375), np.int64(7598)]


In [ ]:
for user in test_data['user_id'].unique()[:5]:
    recommended_books = recommend_books_for_user(user, N=10)
    print(f"User {user} top-10 recommendations: {recommended_books}")

User 51243 top-10 recommendations: [np.int64(6368), np.int64(6374), np.int64(4712), np.int64(4396), np.int64(5805), np.int64(8332), np.int64(3562), np.int64(4775), np.int64(5375), np.int64(7598)]
User 33890 top-10 recommendations: [np.int64(5518), np.int64(6333), np.int64(4810), np.int64(7644), np.int64(8464), np.int64(3699), np.int64(4286), np.int64(6139), np.int64(5561), np.int64(3857)]
User 40972 top-10 recommendations: [np.int64(5337), np.int64(7722), np.int64(5004), np.int64(7466), np.int64(3668), np.int64(9115), np.int64(7741), np.int64(8171), np.int64(8624), np.int64(4349)]
User 27273 top-10 recommendations: [np.int64(3975), np.int64(4817), np.int64(4448), np.int64(4954), np.int64(4409), np.int64(5811), np.int64(4761), np.int64(5148), np.int64(5190), np.int64(6111)]
User 42938 top-10 recommendations: [np.int64(4901), np.int64(5721), np.int64(3514), np.int64(4479), np.int64(5410), np.int64(3527), np.int64(7818), np.int64(3875), np.int64(5744), np.int64(4816)]
